In [134]:
import os

In [2]:
glove_path = 'D:\Work\Project\Project.DataScience\model\glove\glove.840B.300d.txt'

In [135]:
glove_path = '/mnt/DATA/glove.6B/glove.6B.300d.txt'

## 思路
1.从glove文件生成一个word和偏移量的索引
2.根据任务生成一个vocab
3.根据vocab生成tokenizer和embedding

In [136]:
TEST_EMBEDDING_FILE = 'D:\Work\Project\Project.DataScience\model\glove\embedding.txt'

In [137]:
TEST_EMBEDDING_FILE = 'embedding.txt'

In [138]:
# 生成测试用的文件
NUM_LINE = 100
lines = []
with open(glove_path, "r") as fp:
    for i in range(NUM_LINE):
        lines.append(fp.readline())

with open(TEST_EMBEDDING_FILE, 'w') as fp:
    fp.write(''.join(lines))

In [139]:
# 保存文件
import pickle


def _save_var(var, filepath):
    with open(file=filepath, mode="wb") as fp:
        pickle.dump(var, file=fp)
    
    
def _load_var(filepath):
    with open(file=filepath, mode="rb") as fp:
        return pickle.load(fp)



In [140]:
import tqdm
import os

In [141]:
# 生成 index
def _index(embedding):
    index = {}
    with open(embedding, 'r', encoding='utf-8') as fp:
        linecnt = 0
        word = ''
        getword = False
        fp.seek(0, 2)
        filesize = fp.tell()
        fp.seek(0, 0)
        bar_format = "{percentage:3.0f}% | {n_fmt}/{total_fmt} | [{elapsed}<{remaining}]"
        with tqdm.tqdm(total=filesize, bar_format=bar_format) as p_bar:
            while True:
                c = fp.read(1)
                
                # 如果不是最后一个字符
                if c is not "":
                    linecnt += 1
                    
                # word为第一个空格前的字符           
                if getword is False:
                    if c == " ":
                        getword = True
                    else:
                        word = word + c
                
                # 如果已经是最后一个字符, 或者遇到回车符
                if c == '\n' or c is "":
                    if linecnt is not 0:
                        index[word] = (fp.tell() - linecnt, linecnt)
                    p_bar.update(linecnt)
                    linecnt = 0
                    getword = False
                    word = ''
                
                if c is "":
                    break
   
    '''
    # 下面这段代码有问题, 每一行的大小都少算一个字节
    with open(TEST_EMBEDDING_FILE, 'r') as fp:
        for line in fp.readlines():
            items = line.split(" ")
            word = items[0]
            index[word] = (size_cnt, len(line))
            size_cnt = size_cnt + len(line)
    '''
    
    return index


index = _index(TEST_EMBEDDING_FILE)


def _check_index(embedding, index):
    a = {}
    b = {}
    with open(embedding, 'r') as fp:
        for line in fp.readlines():
            items = line.split(" ")
            a[items[0]] = line
    with open(embedding, 'r') as fp: 
        for k in index:
            fp.seek(index[k][0], 0)
            b[k] = fp.read(index[k][1])
    
    cont = 0
    for k in a:
        if a[k] != b[k]:
            print("wrong: " + k)
            print(a[k])
            print(b[k])
        else:
            cont += 1

    print(cont)
    
    
_check_index(TEST_EMBEDDING_FILE, index)

  0% | 0/260340 | [00:00<?]

100% | 260340/260340 | [00:00<00:00]

100


In [142]:
_save_var(index, "123")

In [143]:
index_l = _load_var("123")

In [144]:
# print(index_l)

In [145]:
embedding = TEST_EMBEDDING_FILE

def load_index():
    index_file = embedding + ".index"
    return _load_var(index_file)

def save_index(index):
    index_file = embedding + ".index"
    _save_var(index, index_file)


In [146]:
save_index(index)

In [147]:
index_2 = load_index()

In [149]:
wrong_cnt = 0
correct_cnt = 0
total = 0
for key in index:
    if index[key] != index[key]:
        wrong_cnt += 1
    else:
        correct_cnt += 1
    total += 1
print("correct %d, wrong %d, total %d" %(correct_cnt, wrong_cnt, total))

correct 100, wrong 0, total 100


In [150]:
# 读取vector
m_index = index
m_embedding_fp = open(embedding, 'r')


def _load_vec(word):
    pos = m_index[word]
    m_embedding_fp.seek(pos[0])
    line = m_embedding_fp.read(pos[1])
    items = line.split(" ")
    word_in_embedding_file = items[0]
    if word_in_embedding_file != word:
        raise Exception("index not correct")
    return list(map(float, items[1:]))

In [170]:
m_token = {}

In [171]:
# 根据vocab加载embedding




def _build_token(vocab):
    for i, word in enumerate(vocab):
        m_token[word] = i
    return m_token

def build(vocab, cache=None):
    if cache is not None:
        m_my_embedding = cache
        m_my_embedding_token = cache + ".token"
        
        if os.path.exists(cache):
            my_token = _load_var(m_my_embedding_token)
            new_word = set(vocab) - set(my_token.keys())
            print(my_token)
            print(new_word)

        else:
            new_word = set(vocab)
        
        if len(new_word) > 0:
            embedding_vec = list([_load_vec(word) for word in vocab])
            m_token = _build_token(vocab)
            _save_var(m_token, m_my_embedding_token)
            _save_var(embedding_vec, m_my_embedding)
            
        else:
            embedding_vec = _load_var(m_my_embedding)
            
    else:
        embedding_vec = list([_load_vec(word) for word in vocab])
                    
    return embedding_vec


def tokenize(word):
    return m_token[word]


In [172]:
vocab = ['of', 'to', 'her']

In [178]:
vec = build(vocab, 'test1')

{'of': 0, 'to': 1, 'and': 2, 'you': 3, 'her': 2}
set()


In [179]:
print(len(vec))

3


In [180]:
print(len(vec))
print(m_token)

3
{}


In [181]:
print(tokenize('of'))

KeyError: 'of'